In [1]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import calinski_harabasz_score, davies_bouldin_score

# Functions

In [2]:
def runKmeans_n2(data, score):
    '''
    Runs the k-means for a given cohort with 2 clusters. It predicts the labels of new values and uses the information to get a cutoff.
    It also saves the silhoette coefficient, inertia, calinsiki index and davies index
    Input:
    data (dataframe): HRD results, the rows are different samples and the columns contains information about the sample as well as the the 4 HRD scores
    score (string): Indicates for which score a cutoff should be calculated
    Output:
    results (dictionary): Results contains 5 keys, does are silhouette_coef (average silhouette coefficient), inertia , calinski (Calinski index), davies (Davies index) and n2_cutoff (cutoff value between HRD-high and HRD-low)
    '''
    k = 2
    results = dict()
    
    
    data_hrd = data[score].to_numpy().reshape(-1, 1)
    np.random.seed(42)
    kmeans = KMeans(n_clusters=k, max_iter = 1000)

    # Fit the data to the model
    kmeans.fit(data_hrd)
    
    labels_data = kmeans.predict(data_hrd)
    
    # Get silhouette coefficient
    silhouette_coef = silhouette_score(data_hrd, labels_data)
    inertia = kmeans.inertia_
    
    # Calculate the Calinski-Harabasz index
    calinski_score = calinski_harabasz_score(data_hrd, labels_data)

    # Calculate the Davies-Bouldin index
    davies_score = davies_bouldin_score(data_hrd, labels_data)
    
    # Predict the labels for values between 0 and max(HRDsum)
    max_number = int(max(list(data[score])))
    numbers_vector = np.arange(max_number + 1).reshape(-1, 1)
    labels_numbers = kmeans.predict(numbers_vector).reshape(-1, 1)

    # Find the minimum value for each label
    min_values = [np.min(numbers_vector[labels_numbers == label]) for label in np.unique(labels_numbers)]

    results['silhouette_coef'] = silhouette_coef
    results['inertia'] = inertia
    results['calinski'] = calinski_score
    results['davies'] = davies_score
    results['n2_cutoff'] = int(max(min_values))
    
    return results

def runKmeans_pancancer_n2(data, score):
    '''
    Runs the K-means for each cancer type in the pan cancer cohort with 2 clusters and saves the results in a dataframe
    Input:
    data (dataframe): HRD results, the rows are different samples and the columns contains information about the sample as well as the the 4 HRD scores
    score (string): Indicates for which score a cutoff should be calculated
    Output:
    df (dataframe): Cotains the results from the K-means (columns) for all cohorts (rows). The columns include the total number of cases per cohort and the number of case per type (HRD-high, HRD-low, HRD-medium)
    '''
    
    ## Prepare dataframe
    df = pd.DataFrame(columns=['Project ID','n_cases', 'n2_cutoff', 'n2_silhouette_coef', 'n2_inertia', 'n2_calinski','n2_davies','n2_n_low','n2_n_high'])
    types = np.unique(data['Project ID'])

    for cancer_type in types:
        ## Run K-means
        sub_data = data[data['Project ID'] == cancer_type]
        results = runKmeans_n2(sub_data, score)
        
        ## Extract number of sample per type (high, low)
        n2_cutoff = results['n2_cutoff']
        high = sub_data[sub_data[score]>=n2_cutoff]
        low = sub_data[sub_data[score]<n2_cutoff]

        df.loc[len(df)] = [cancer_type, sub_data.shape[0], n2_cutoff, results['silhouette_coef'], results['inertia'], results['calinski'], results['davies'], high.shape[0],  low.shape[0]]
        
    return df

def runKmeans_n3(data, score):
    '''
    Runs the k-means for a given cohort with 3 clusters. It predicts the labels of new values and uses the information to get a cutoff.
    It also saves the silhoette coefficient, inertia, calinsiki index and davies index
    Input:
    data (dataframe): HRD results, the rows are different samples and the columns contains information about the sample as well as the the 4 HRD scores
    score (string): Indicates for which score a cutoff should be calculated
    Output:
    results (dictionary): Results contains 6 keys, does are silhouette_coef (average silhouette coefficient), inertia , calinski (Calinski index), davies (Davies index) and n3_cutoff_lm (cutoff value between HRD-low and HRD-medium)
    and n3_cutoff_mh (cutoff value between HRD-medium and HRD-high)
    '''    
    k = 3
    results = dict()
    
    
    data_hrd = data[score].to_numpy().reshape(-1, 1)
    np.random.seed(42)
    kmeans = KMeans(n_clusters=k, max_iter = 1000)

    # Fit the data to the model
    kmeans.fit(data_hrd)
    
    labels_data = kmeans.predict(data_hrd)
    
    # Get silhouette coefficient
    silhouette_coef = silhouette_score(data_hrd, labels_data)
    inertia = kmeans.inertia_
    
    # Calculate the Calinski-Harabasz index
    calinski_score = calinski_harabasz_score(data_hrd, labels_data)

    # Calculate the Davies-Bouldin index
    davies_score = davies_bouldin_score(data_hrd, labels_data)
    
    # Predict the labels for values between 0 and max(HRDsum)
    max_number = int(max(list(data[score])))
    numbers_vector = np.arange(max_number + 1).reshape(-1, 1)
    labels_numbers = kmeans.predict(numbers_vector).reshape(-1, 1)

    # Find the minimum value for each label
    min_values = [np.min(numbers_vector[labels_numbers == label]) for label in np.unique(labels_numbers)]
    
    min_values.sort()
    results['silhouette_coef'] = silhouette_coef
    results['inertia'] = inertia
    results['calinski'] = calinski_score
    results['davies'] = davies_score
    results['n3_cutoff_lm'] = int(min_values[-2])
    results['n3_cutoff_mh'] = int(min_values[-1])
    
    return results

def runKmeans_pancancer_n3(data, score):
    '''
    Runs the K-means for each cancer type in the pan cancer cohort with 3 clusters and saves the results in a dataframe
    Input:
    data (dataframe): HRD results, the rows are different samples and the columns contains information about the sample as well as the the 4 HRD scores
    score (string): Indicates for which score a cutoff should be calculated
    Output:
    df (dataframe): Cotains the results from the K-means (columns) for all cohorts (rows). The columns include the total number of cases per cohort and the number of case per type (HRD-high, HRD-low, HRD-medium)
    '''
    
    ## Prepare dataframe
    df = pd.DataFrame(columns=['Project ID','n_cases', 'n3_cutoff_lm','n3_cutoff_mh', 'n3_silhouette_coef', 'n3_inertia', 'n3_calinski', 'n3_davies','n_3_n_high','n3_n_medium','n3_n_low'])
    types = np.unique(data['Project ID'])

    for cancer_type in types:
        
        ## Run K-means
        sub_data = data[data['Project ID'] == cancer_type]
        results = runKmeans_n3(sub_data, score)
        
        ## Extract number of sample per type (high, medium, low)
        cutoff_lm = results['n3_cutoff_lm']
        cutoff_mh = results['n3_cutoff_mh']
        high = sub_data[sub_data[score]>=cutoff_mh]
        low = sub_data[sub_data[score]<cutoff_lm]
        medium = sub_data[(sub_data[score]<cutoff_mh) & (sub_data[score]>=cutoff_lm)]
        df.loc[len(df)] = [cancer_type, sub_data.shape[0], cutoff_lm, cutoff_mh, results['silhouette_coef'], results['inertia'], results['calinski'], results['davies'], high.shape[0], medium.shape[0], low.shape[0]]
        
    return df


def analyseStatistics(data):
    '''
    Function that runs the k-means for all cohorts with 2 to 6 clusters and saves the statsitics (avg. silhouette coef., inertia, calinski index, davies index).
    Input:
    data (dataframe): HRD results, the rows are different samples and the columns contains information about the sample as well as the the 4 HRD scores
    Output:
    df (dataframe): Containg all the results for each cohort for each number of clusters (rows are the different cohorts, columns are the different metrics)
    '''
    
    n_silhoette = [f"n{x}_silhouette" for x in range(2, 7)]
    n_inertia = [f"n{x}_inertia" for x in range(2, 7)]
    n_calinski = [f"n{x}_calinski" for x in range(2, 7)]
    n_davies = [f"n{x}_davies" for x in range(2, 7)]
    
    column_names = ['Project ID'] + n_silhoette + n_inertia + n_calinski + n_davies
    
    df = pd.DataFrame(columns = column_names)
    types = np.unique(data['Project ID'])
    clusters = [2,3,4,5,6]
    
    for cancer_type in types:
        results_type = dict()
        ## Subset data for a single cohort
        sub_data = data[data['Project ID'] == cancer_type]
        data_hrd = sub_data['HRD_sum'].to_numpy().reshape(-1, 1)

        for k in clusters:
            
            ## Runs k-means for n_cluster = 2 to 6 and save the different metrics
            np.random.seed(42)
            kmeans = KMeans(n_clusters=k, max_iter = 1000).fit(data_hrd)
            labels_data = kmeans.predict(data_hrd)
            results_type[str(k)+'_silhouette'] = silhouette_score(data_hrd, labels_data)
            results_type[str(k)+'_inertia'] = kmeans.inertia_
            results_type[str(k)+'_calinski'] = calinski_harabasz_score(data_hrd, labels_data)
            results_type[str(k)+'_davies'] = davies_bouldin_score(data_hrd, labels_data)
            print('Finished analysis for n_comp = '+ str(k) + ' for cohort '+ cancer_type)
        df.loc[len(df)] = [cancer_type] + [results_type[str(k)+'_silhouette'] for k in clusters] + [results_type[str(k)+'_inertia'] for k in clusters] + [results_type[str(k)+'_calinski'] for k in clusters] + [results_type[str(k)+'_davies'] for k in clusters]
        print('Finished cohort '+ cancer_type)
    df.to_csv('../data/kmeans_cluster_analysis.csv', sep=',', header = True)
    
    return df

def loadData(path_data_file):
    '''
    Loads the data and subset it to only extract the primary type.
    Input:
    path_data_file (string): Full path and name of the datafile containing at least the columns (Project ID, HRD_sum, Type)
    Output:
    primary (dataframe): Data containing only the primary sample type
    '''
    HRD_pan_cancer = pd.read_csv(path_data_file, sep=',', header = 0)
    primary = HRD_pan_cancer[HRD_pan_cancer['Type'] == 'Primary']
    primary = primary[primary['Project ID'] != 'TARGET-CCSK']
    
    return primary

def runKmeansCutoff(mode, path_data_file, path_results, score, filename = 'kmeans_cutoff_pancancer'):
    '''
    Runs K-means to infer cutoff.
    Input:
    mode (string): Indicates the number of clusters 'n2' for 2 clusters (low and high), 'n3' for 3 clusters (low, medium, high) or 'both' to get the results for 2 and 3 clusters
    path_data_file (string): Full path and name of the datafile containing at least the columns (Project ID, HRD_sum, Type)
    path_results (string): Path to the folder where the output should be saved
    score (string): Indicates for which score a cutoff should be calculated
    filename (string): Name of the outputfile (default: kmeans_cutoff_pancancer)
    '''
    data = loadData(path_data_file)
    
    if mode == 'n2':
        results_n2 = runKmeans_pancancer_n2(data, score)
        results_n2.to_csv(path_results + '/' + score + '_' + mode+ '_' + filename, sep=',', header = True)
    elif mode == 'n3':
        results_n3 = runKmeans_pancancer_n3(data, score)
        results_n3.to_csv(path_results + '/' + score + '_' + mode + '_' + filename, sep=',', header = True)
    else:
        results_n2 = runKmeans_pancancer_n2(data, score)
        results_n3 = runKmeans_pancancer_n3(data, score)
        results = pd.merge(results_n2, results_n3, on=['Project ID','n_cases'])
        results.to_csv(path_results + '/' + score + '_' + filename + '.csv', sep=',', header = True)
        


In [13]:
HRD_pan_cancer = pd.read_csv('../../HRD_score/data/HRD_scores_pan_cancer_annotated_v2.csv', sep=',', header = 0)
primary = HRD_pan_cancer[HRD_pan_cancer['Type'] == 'Primary']

In [16]:
results_n2 = runKmeans_pancancer_n2(primary)
results_n3 = runKmeans_pancancer_n3(primary)

In [17]:
results = pd.merge(results_n2, results_n3, on=['Project ID','n_cases'])

In [18]:
results.to_csv('../data/kmeans_cutoffs_pancancer.csv', sep=',', header = True)

In [33]:
analyseStatistics(primary)

Empty DataFrame
Columns: [Project ID, n2_silhouette, n3_silhouette, n4_silhouette, n5_silhouette, n6_silhouette, n2_inertia, n3_inertia, n4_inertia, n5_inertia, n6_inertia, n2_calinski, n3_calinski, n4_calinski, n5_calinski, n6_calinski, n2_davies, n3_davies, n4_davies, n5_davies, n6_davies]
Index: []

[0 rows x 21 columns]
Finished analysis for n_comp = 2 for cohort TARGET-ALL-P2
Finished analysis for n_comp = 3 for cohort TARGET-ALL-P2
Finished analysis for n_comp = 4 for cohort TARGET-ALL-P2
Finished analysis for n_comp = 5 for cohort TARGET-ALL-P2
Finished analysis for n_comp = 6 for cohort TARGET-ALL-P2
Finished cohort TARGET-ALL-P2
Finished analysis for n_comp = 2 for cohort TARGET-AML
Finished analysis for n_comp = 3 for cohort TARGET-AML
Finished analysis for n_comp = 4 for cohort TARGET-AML
Finished analysis for n_comp = 5 for cohort TARGET-AML
Finished analysis for n_comp = 6 for cohort TARGET-AML
Finished cohort TARGET-AML
Finished analysis for n_comp = 2 for cohort TARGET-C

Finished analysis for n_comp = 6 for cohort TCGA-PRAD
Finished cohort TCGA-PRAD
Finished analysis for n_comp = 2 for cohort TCGA-READ
Finished analysis for n_comp = 3 for cohort TCGA-READ
Finished analysis for n_comp = 4 for cohort TCGA-READ
Finished analysis for n_comp = 5 for cohort TCGA-READ
Finished analysis for n_comp = 6 for cohort TCGA-READ
Finished cohort TCGA-READ
Finished analysis for n_comp = 2 for cohort TCGA-SARC
Finished analysis for n_comp = 3 for cohort TCGA-SARC
Finished analysis for n_comp = 4 for cohort TCGA-SARC
Finished analysis for n_comp = 5 for cohort TCGA-SARC
Finished analysis for n_comp = 6 for cohort TCGA-SARC
Finished cohort TCGA-SARC
Finished analysis for n_comp = 2 for cohort TCGA-SKCM
Finished analysis for n_comp = 3 for cohort TCGA-SKCM
Finished analysis for n_comp = 4 for cohort TCGA-SKCM
Finished analysis for n_comp = 5 for cohort TCGA-SKCM
Finished analysis for n_comp = 6 for cohort TCGA-SKCM
Finished cohort TCGA-SKCM
Finished analysis for n_comp = 2

,Project ID,n2_silhouette,n3_silhouette,n4_silhouette,n5_silhouette,n6_silhouette,n2_inertia,n3_inertia,n4_inertia,n5_inertia,...,n2_calinski,n3_calinski,n4_calinski,n5_calinski,n6_calinski,n2_davies,n3_davies,n4_davies,n5_davies,n6_davies
0,TARGET-ALL-P2,0.623348,0.593944,0.663001,0.688384,0.705436,942.302952,518.453612,264.577302,165.374619,...,600.343261,662.236230,954.579155,1184.626422,1640.962253,0.548829,0.554489,0.454316,0.457863,0.422265
1,TARGET-AML,0.899806,0.760432,0.753573,0.750000,0.757173,401.757576,175.892241,77.892241,26.305769,...,209.088630,276.906518,437.291216,986.837607,1510.930368,0.336955,0.388936,0.218311,0.274342,0.280967
2,TARGET-CCSK,0.520960,0.591486,0.631818,0.681818,0.727273,12.000000,3.950000,1.416667,0.666667,...,17.181818,31.350978,55.163993,77.045455,1.000000,0.619048,0.512045,0.354010,0.231429,0.000000
3,TARGET-OS,0.561321,0.534790,0.582138,0.557661,0.552139,6720.200000,3738.523932,1970.119596,1276.348293,...,127.639981,144.372372,203.339058,242.669955,246.085697,0.604244,0.557685,0.504498,0.501876,0.422759
4,TCGA-ACC,0.583560,0.648230,0.595537,0.598910,0.560248,7650.413793,2663.805556,1809.473333,1192.940000,...,130.327374,266.453955,272.034737,316.854366,448.924921,0.624738,0.448023,0.512863,0.415149,0.456644
5,TCGA-BLCA,0.595380,0.587023,0.576744,0.554251,0.579510,44579.408069,21273.617693,12164.876166,8031.933948,...,825.288511,1075.015602,1346.667797,1575.413356,1920.832659,0.547042,0.512920,0.505659,0.522908,0.474808
6,TCGA-BRCA,0.625439,0.599230,0.576598,0.554939,0.548625,154023.627311,66484.389812,39140.671164,25450.594704,...,2667.422871,3791.337500,4538.235981,5373.266537,6211.836601,0.529391,0.495137,0.506014,0.519384,0.514757
7,TCGA-CESC,0.607730,0.519853,0.562866,0.549177,0.524295,17719.389583,10295.256762,5336.754899,3598.118013,...,578.872425,603.203658,864.512300,993.943991,1056.829226,0.557901,0.578985,0.500986,0.518374,0.531675
8,TCGA-CHOL,0.646565,0.587959,0.613250,0.664733,0.679788,960.857143,439.444444,221.064528,106.241667,...,75.937490,100.155626,139.244984,218.887906,279.144678,0.526648,0.517420,0.411729,0.383083,0.378976
9,TCGA-COAD,0.593490,0.598273,0.592416,0.599890,0.596876,23128.705646,11355.109666,6946.503266,4364.344182,...,809.494694,1042.284786,1222.654955,1518.496517,1880.154177,0.577391,0.506081,0.494812,0.452149,0.459455


# Run K-means cutoff

In [4]:
runKmeansCutoff(mode = 'both', path_data_file = '../../HRD_score/data/HRD_scores_pan_cancer_annotated_v2.csv', path_results = '../data', score = 'TAI', filename = 'kmeans_cutoff_pancancer')